<a href="https://colab.research.google.com/github/MihalkaProMuzon/RAG-assistant/blob/main/C%D0%B1%D0%BE%D1%80%D0%BA%D0%B0_%D0%BF%D1%80%D0%B8%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Сетап

In [1]:
#@title Установка библиотек в среду ( ! Требуется перезапуск среды )

from IPython.display import clear_output
!pip install llama-index llama-index-llms-deepseek chromadb sentence-transformers llama-index-embeddings-huggingface unstructured gdown docx2txt "arize-phoenix[evals,llama-index]" gcsfs nest-asyncio "openinference-instrumentation-llama-index>=2.0.0" nemoguardrails langchain-openai llama-index-postprocessor-colbert-rerank
clear_output()

In [1]:
# @title Ключ VseGPT
import os
import getpass

os.environ["VSEGPT_API_KEY"] = getpass.getpass("API ключ от VseGPT сервиса: ")

API ключ от VseGPT сервиса: ··········


In [2]:
#@title Загрузка тестового набора документов

!test -d "Документы" && rm -Rf "Документы"
!gdown 1u1fEfsMOOu4ll5XeTS9SOT7gXQDN5Rkb
import zipfile
with zipfile.ZipFile("Документы.zip", 'r') as zip_ref:
    zip_ref.extractall()
!rm "Документы.zip"

Downloading...
From: https://drive.google.com/uc?id=1u1fEfsMOOu4ll5XeTS9SOT7gXQDN5Rkb
To: /content/Документы.zip
100% 2.30M/2.30M [00:00<00:00, 70.4MB/s]


## Код приложения

In [3]:
# @title Импорт библиотек

import hashlib
import io
import json
import os
import re
import threading
import time
import zipfile
from pathlib import Path
from typing import Any, Dict, List, Mapping, Optional, Pattern, Tuple
from urllib.parse import unquote, urlparse

import chromadb
import nest_asyncio
import phoenix as px
import requests
import traceback
import asyncio

# IPython и Google Colab
from google.colab.output import eval_js
from IPython.display import IFrame, display

# LlamaIndex основные компоненты
from llama_index.core import (
    Document,
    GPTVectorStoreIndex,
    QueryBundle,
    Settings,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.core.schema import TextNode
from llama_index.core.base.llms.types import (
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
from llama_index.core.llms import CustomLLM
from llama_index.core.llms.callbacks import llm_completion_callback
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import TextNode
from llama_index.core.storage.storage_context import StorageContext

# LlamaIndex интеграции
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.deepseek import DeepSeek
from llama_index.postprocessor.colbert_rerank import ColbertRerank

# LangChain
from langchain.llms.base import BaseLLM
from langchain.schema import Generation, LLMResult

# NeMo Guardrails
from nemoguardrails import LLMRails, RailsConfig
from nemoguardrails.actions import action
from nemoguardrails.llm.providers import register_llm_provider
from nemoguardrails.llm.types import Task

# Phoenix и OpenTelemetry
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations

# Flask
from flask import Flask, jsonify, redirect, render_template, request, send_file, url_for

# Дополнительные утилиты
import importlib.util

nest_asyncio.apply()

In [4]:
#@title Объявление директорий
!mkdir templates
!mkdir static
!mkdir static/css
!mkdir static/js

!mkdir guardrails
!mkdir guardrails/rails

In [5]:
#@title helper.js
%%writefile static/js/helper.js

function docelm(name) {
    return document.getElementById(name)
}

function docelmstyle(name) {
    return document.getElementById(name).style
}

function context_formatter(txt, start, end, context_lenght){
    let before = txt.slice(0, start);
    let center = txt.slice(start, end);
    let after = txt.slice(end, txt.length-1);

    let formatted = `${before}<b><i>${center}</i></b>${after}`
    let formatted_len = formatted.length

    let text_start = Math.max(start - context_lenght-6, 0);
    let text_end = Math.min(end + 8 + context_lenght, formatted_len);
    let context = `${formatted.slice(text_start, text_end)}`

    if (text_start != 0){
        context = `... ${context}`
    }
    if (text_end != formatted_len){
        context = `${context} ...`
    }

    return context
}

function topics_formatter(topics_dict){
    topics = `<p>`
    Object.entries(topics_dict).forEach(([key, value]) => {
        toper = '#'.repeat(key);
        topics = topics + `${toper}${value}<br>`
    });
    topics = topics + `</p>`
    return topics
}

Writing static/js/helper.js


### Загрузка документов в хранилище

In [6]:
#@title templates/fileloader (html Индексирование документов)
%%writefile templates/fileloader.html


<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>File Indexer</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
    <script src="{{ url_for('static', filename='js/helper.js') }}"></script>

    <div class="container">
        <form action="/">
            <button id="load-docs-btn" type="submit">На главную</button>
        </form>
        <h1>📁 Индекс файлов</h1>

        <div class="input-group">
            <input type="text" id="folderPath" placeholder="Введите путь к папке..." value="Документы">
            <button class="btn-primary" onclick="scanFolder()">Сканировать</button>
        </div>

        <div id="error" class="error" style="display: none;"></div>

        <div id="fileTree" class="file-tree" style="display: none;">
            <!-- Дерево файлов будет здесь -->
        </div>

        <div style="text-align: center;">
            <button id="indexBtn" class="btn-success" onclick="checkFilesToIndex()" style="display: none;">
                Индексировать выбранные файлы
            </button>
        </div>

        <div id='message' style="text-align: center;"></div>

        <!-- Устранение замечаний -->

        <div id='fichesBlock' class="container" style="display: none;">
            <h1>🎯 Устранение замечаний</h1>

            <div class="summary">
                <p style="display: inline-block;"><strong>Всего нюансов: <span id="totalCount">0</strong></span></p>
                <p style="display: inline-block;">Решено: <span id="resolvedCount">0</span></p>
                <p style="display: inline-block;">Осталось: <span id="remainingCount">0</span></p>
            </div>

            <div class="controls">
                <button id="resetBtn" onclick="resetAll()">🔄 Сбросить все</button>
                <button id="submitBtn" onclick="submitFixes()">🚀 Применить исправления</button>
            </div>

            <div id="fichesContainer" class="fiches-container">
                <!-- Устраняемые замечания будут добавляться здесь -->
            </div>

        </div>
    </div>

    <script src="static/js/fileloader_selectDocs.js"></script>
    <script src="static/js/fileloader_fichesResolve.js"></script>
</body>
</html>

Writing templates/fileloader.html


In [7]:
#@title static/js/fileloader_selectDocs (js  Выбор документов)
%%writefile static/js/fileloader_selectDocs.js



let currentTree = null;

function scanFolder() {
    const path = document.getElementById('folderPath').value.trim();
    if (!path) {
        showError('Введите путь к папке');
        return;
    }

    fetch('/scan', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json',
        },
        body: JSON.stringify({ path: path })
    })
    .then(response => response.json())
    .then(data => {
        if (data.status === "error") {
            showError(data.message);
            return;
        }

        currentTree = data.tree;
        renderFileTree(currentTree);
        document.getElementById('fileTree').style.display = 'block';
        document.getElementById('indexBtn').style.display = 'block';
        document.getElementById('error').style.display = 'none';
        document.getElementById('message').style.display = 'none';
        document.getElementById('fichesBlock').style.display = 'none';
    })
    .catch(error => {
        showError('Ошибка при сканировании: ' + error);
    });
}

function renderFileTree(tree, parentElement = null) {
    const container = parentElement || document.getElementById('fileTree');
    container.innerHTML = '';

    const renderItem = (item) => {
        const itemDiv = document.createElement('div');
        itemDiv.className = 'tree-item';
        itemDiv.style.paddingLeft = '15px'

        if (item.type === 'folder') {
            const childrenDiv = document.createElement('div');
            childrenDiv.style.display = item.expanded ? 'block' : 'none'

            itemDiv.innerHTML = `
                <div>
                    <span class="toggle-btn" onclick="toggleFolder(this, '${item.path}')">${item.expanded ? '▼' : '▶'}</span>
                    <input type="checkbox" class="checkbox" onchange="doSelection(this, '${item.path}')" ${item.selected ? 'checked' : ''}>
                    <span class="folder">📁 ${item.name}</span>
                </div>
            `;

            // Рекурсивно рендерим детей
            item.children.forEach(child => {
                childrenDiv.appendChild(renderItem(child));
            });

            itemDiv.appendChild(childrenDiv);
        } else {
            itemDiv.innerHTML = `
                <div class="file">
                    <input type="checkbox" class="checkbox" onchange="doSelection(this, '${item.path}')" ${item.selected ? 'checked' : ''}>
                    <span>📄 ${item.name}</span>
                </div>
            `;
        }

        return itemDiv;
    };

    container.appendChild(renderItem(tree));
}

function toggleFolder(btn, folderPath) {
    const children = btn.parentElement.nextElementSibling;

    // Обновляем отображение
    const isExpanded = children.style.display === 'block';
    children.style.display = isExpanded ? 'none' : 'block';
    btn.textContent = isExpanded ? '▶' : '▼';

    // Обновляем состояние в данных дерева
    const updateExpandedState = (item) => {
        if (item.path === folderPath) {
            item.expanded = !isExpanded;
        }
        if (item.type == 'folder'){
            item.children.forEach(child => updateExpandedState(child));
        }
    };
    updateExpandedState(currentTree);
}

function deselect_item(item_path){
    const deselect = (item, path) => {
        if (item.path === path) {
            item.selected = false;
            if (item.parent_path != null){
                deselect(currentTree, item.parent_path)
            }
        }
        if (item.type == 'folder'){
            item.children.forEach(child => deselect(child, path));
        }
    };
    deselect(currentTree, item_path);
}
function doSelection(checkbox, filePath) {
    const setFileSelection = (item, isSelected, path) => {
        if (item.path === path) {
            item.selected = isSelected
            if (item.type === 'folder') {
                item.children.forEach(child => setFileSelection(child, isSelected, child.path));
            }
            else if(!isSelected){
                deselect_item(item.parent_path)
            }
        }
        if (item.type === 'folder') {
            item.children.forEach(child => setFileSelection(child, isSelected, path));
        }
    };
    setFileSelection(currentTree, checkbox.checked, filePath);
    renderFileTree(currentTree);
}

function checkFilesToIndex() {
    if (!currentTree) {
        showError('Нет данных для индексации');
        return;
    }

    docelmstyle('message').display = 'block';
    docelm("message").innerHTML = "Парсим данные";

    fetch('/check_files_to_index', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json',
        },
        body: JSON.stringify({ tree: currentTree })
    })
    .then(response => response.json())
    .then(data => {
        if (data.status === "error") {
            docelm("message").innerHTML = "";
            showError(data.message);
            return;
        }

        docelmstyle('fichesContainer').display = 'block';
        docelmstyle('fileTree').display = 'none';
        docelmstyle('indexBtn').display = 'none';

        loadFiches(data);
    })
    .catch(error => {
        showError('Ошибка при индексации: ' + error);
    });
}

function showError(message) {
    const errorDiv = document.getElementById('error');
    errorDiv.textContent = message;
    errorDiv.style.display = 'block';
}

Writing static/js/fileloader_selectDocs.js


In [8]:
#@title static/js/fileloader_fichesResolve (js Устранение замечаний)
%%writefile static/js/fileloader_fichesResolve.js

// Оригинальные данные с сервера
let originalData = [];
// Текущие данные с решениями пользователя
let currentFiches = [];

// Отмена индексации документов
const cancelledDocs = new Map();

// Загрузка данных с сервера
function loadFiches(data) {
    originalData = data
    currentFiches = JSON.parse(JSON.stringify(originalData['fiches']));

    docelmstyle('message').display = 'none';
    docelm("message").innerHTML = "";

    console.log("*-");
    console.log(originalData);

    renderFiches();
    updateSummary();
}

// Рендеринг всех фич
function renderFiches() {
    docelmstyle("fichesBlock").display = 'block'

    const container = document.getElementById('fichesContainer');
    container.innerHTML = '';

    currentFiches.forEach((fiche, index) => {
        console.log(fiche)
        if (fiche.disabled !== true){
            const ficheElement = createFicheElement(fiche, index);
            container.appendChild(ficheElement);
        }
    });
}

// Создание элемента фичи
function createFicheElement(fiche, index) {
    const div = document.createElement('div');
    div.className = `fiche fiche-${fiche.type} ${fiche.resolved ? 'resolved' : 'pending'}`;

    switch (fiche.type) {
         case 'document_in_index':
            div.innerHTML = createDocumentFiche(fiche, index);
            break;
        case 'link_found':
            div.innerHTML = createLinkFiche(fiche, index);
            break;
        case 'macro_found':
            div.innerHTML = createMacroFiche(fiche, index);
            break;
    }

    return div;
}

// Фича: документ уже в хранилище
function createDocumentFiche(fiche, index) {
    fiche_content_col = fiche.resolved ? "#ddeeff" : "none"

    return `
        <div class="fiche-header">
            <h3>📄 Документ уже в хранилище</h3>
        </div>
        <div class="fiche-content" style="background-color: ${fiche_content_col};">
            <p><strong>Файл:</strong> ${fiche.file_name}</p>

            <div class="actions">
                <select onchange="handleDocumentAction(${index}, this.value)">
                    <option value="" ${!fiche.action ? 'selected' : ''}>Не выбрано</option>
                    <option value="keep" ${fiche.action === 'keep' ? 'selected' : ''}>Индексировать</option>
                    <option value="cancel" ${fiche.action === 'cancel' ? 'selected' : ''}>Отменить индексирование</option>
                </select>
            </div>
        </div>
    `;
}

// Фича: найдена ссылка
function createLinkFiche(fiche, index) {
    const isInMacrosDb = fiche.link.existing_macro;

    fiche_content_col = fiche.resolved ? "#d7e7f7" : "none"

    topics = topics_formatter(fiche.node.topics)
    const txt = originalData.node_texts[fiche.node.id_]
    const link_context = context_formatter(txt, fiche.link.start, fiche.link.end, fiche.fulltext_enabled ? 100000 : 56)
    const link_text = txt.slice(fiche.link.start, fiche.link.end);

    return `
        <div class="fiche-header">
            <h3>🔗 Найдена ссылка ${isInMacrosDb ? '(есть в базе макросов)' : ''}</h3>
        </div>
        <div class="fiche-content" style="background-color: ${fiche_content_col};">
            <p><strong>Файл:</strong> ${fiche.node.file_name}</p>
            ${topics}

            <div class="link-info">
                <div>
                    <p>${link_context}</p>
                </div>
                <button onclick="toggleFullText(${index})"> ${fiche.fulltext_enabled ? "Скрыть текст ноды" : "📖 Показать полный текст ноды"}</button>
                <p>${link_text}</p>
                ${isInMacrosDb ? `<p><strong>Макрос в базе:</strong> ${fiche.link.existing_macro}</p>` : ''}
                <div class="full-text" style="display: none;">${fiche.node.text}</div>
            </div>

            <div class="actions">
                <select onchange="handleLinkAction(${index}, this.value)">
                    <option value="" ${!fiche.action ? 'selected' : ''}>Не выбрано</option>
                    <option value="keep" ${fiche.action === 'keep' ? 'selected' : ''}>Оставить как есть</option>
                    <option value="replace_macro" ${fiche.action === 'replace_macro' ? 'selected' : ''}>Заменить на макрос</option>
                    <option value="replace_text" ${fiche.action === 'replace_text' ? 'selected' : ''}>Заменить на текст</option>
                </select>

                ${fiche.action === 'replace_macro' ? `
                    <div class="macro-input">
                        <input class="nospaces" type="text" value="${fiche.action_value}"
                                placeholder="Имя макроса" onchange="updateReplaceValueNoSpaces(${index}, this.value)">
                    </div>
                ` : ''}

                ${fiche.action === 'replace_text' ? `
                    <div class="text-input">
                        <input type="text" value="${fiche.action_value}"
                                placeholder="Текст для замены" onchange="updateReplaceValue(${index}, this.value)">
                    </div>
                ` : ''}

                <button onclick="applyToAllSimilar('link_found', '${fiche.link.text}', ${index})">
                    🔄 Применить ко всем нерешенным замечаниям с данной ссылкой
                </button>
            </div>
            <div>------------------------------------------------------------------------------------------------------------</div>
        </div>
    `;
}

// Фича: найден макрос
function createMacroFiche(fiche, index) {
    const isInMacrosDb = fiche.macro.existing_macro;

    fiche_content_col = fiche.resolved ? "#c5d0e6" : "none"

    topics = topics_formatter(fiche['node']['topics'])
    const txt = originalData['node_texts'][fiche.node.id_]
    const macro_context = context_formatter(txt, fiche.macro.start, fiche.macro.end, fiche.fulltext_enabled ? 100000 : 56)
    const macro_text = txt.slice(fiche.macro.start, fiche.macro.end);

    const keep_action = isInMacrosDb ? "Игнорировать" : "Добавить в базу макросов"

    return `
        <div class="fiche-header">
            <h3>🔤 Найден макрос ${isInMacrosDb ? '(есть в базе)' : ''}</h3>
        </div>
        <div class="fiche-content" style="background-color: ${fiche_content_col};">
            <p><strong>Файл:</strong> ${fiche.node.file_name}</p>
            ${topics}

            <div class="macro-info">
                <div>
                    <p>${macro_context}</p
                </div>
                <button onclick="toggleFullText(${index})">${fiche.fulltext_enabled ? "Скрыть текст ноды" : "📖 Показать полный текст ноды"}</button>
                <p>${macro_text}</p
                ${isInMacrosDb ? `<p><strong>Значение в базе:</strong> ${fiche.macro.existing_macro}</p>` : ''}
            </div>

            <div class="actions">
                <select onchange="handleMacroAction(${index}, this.value)">
                    <option value="" ${!fiche.action ? 'selected' : ''}>Не выбрано</option>
                    <option value="keep" ${fiche.action === 'keep' ? 'selected' : ''}>${keep_action}</option>
                    <option value="rename" ${fiche.action === 'rename' ? 'selected' : ''}>Изменить имя макроса</option>
                    <option value="replace_text" ${fiche.action === 'replace_text' ? 'selected' : ''}>Заменить на текст</option>
                </select>

                ${fiche.action === 'rename' ? `
                    <div class="macro-input">
                        <input class="nospaces" type="text" value="${fiche.action_value}"
                                placeholder="Новое имя макроса" onchange="updateReplaceValueNoSpaces(${index}, this.value)">
                    </div>
                ` : ''}

                ${fiche.action === 'replace_text' ? `
                    <div class="text-input">
                        <input type="text" value="${fiche.action_value}"
                                placeholder="Текст для замены" onchange="updateReplaceValue(${index}, this.value)">
                    </div>
                ` : ''}

                <button onclick="applyToAllSimilar('macro_found', '${fiche.macro.text}', ${index})">
                    🔄 Применить ко всем замечаниям с данным макросом
                </button>
            </div>
        </div>
    `;
}

// Обработчики действий
function handleDocumentAction(index, action) {
    currentFiches[index].action = action
    currentFiches[index].resolved = action != '';

    currentFiches.forEach((ficha, indx_2)  => {
          if (ficha.type !== "document_in_index"){
              if(ficha.file_name === currentFiches[index]['file_name']){
                  ficha.disabled = currentFiches[index].action === 'cancel'
              }
          }
    })

    renderFiches();
    updateSummary();
}

function handleLinkAction(index, action) {
    currentFiches[index].action = action;
    currentFiches[index].resolved = action !== '';

    if (action === 'replace_text') {
        currentFiches[index].action_value = ''
    }
    else if (action === 'replace_macro') {
        currentFiches[index].action_value = currentFiches[index].link.existing_macro || '';
    }

    updateFiche(index);
}

function handleMacroAction(index, action) {
    currentFiches[index].action = action === '' ? null : action;
    currentFiches[index].resolved = action !== '';

    if (action === 'replace_text') {
        currentFiches[index].action_value = ''
    }
    else if (action === 'rename') {
        currentFiches[index].action_value = currentFiches[index].macro.macro_name || '';
    }

    updateFiche(index);
}

function updateFiche(index) {
    const container = document.getElementById('fichesContainer');
    const newElement = createFicheElement(currentFiches[index], index);
    const oldElement = container.children[index];
    container.replaceChild(newElement, oldElement);
    updateSummary();
}

function updateSummary() {
    const enabledFiches = currentFiches.filter(f => !f.disabled);
    const total = enabledFiches.length
    const resolved = enabledFiches.filter(f => f.resolved).length;

    document.getElementById('totalCount').textContent = total;
    document.getElementById('resolvedCount').textContent = resolved;
    document.getElementById('remainingCount').textContent = total - resolved;
}

function toggleFullText(index) {
    if (currentFiches[index].fulltext_enabled){
        currentFiches[index].fulltext_enabled = false;
    }
    else{
        currentFiches[index].fulltext_enabled = true;
    }
    updateFiche(index);
}
function updateReplaceValue(index, value) {
    currentFiches[index].action_value = value
}
function updateReplaceValueNoSpaces(index, value) {
    currentFiches[index].action_value = value.replace(/\s+/g, '_');
}
document.addEventListener('input', function(event) {
    if (event.target.classList.contains('nospaces')) {
        event.target.value = event.target.value.replace(/\s+/g, '_');
    }
});

function applyToAllSimilar(type, value, sourceIndex) {
    orig_ficha = currentFiches[sourceIndex]
    switch (type) {
        case "link_found":
            currentFiches.forEach((ficha, index) => {
                if (ficha.type === "link_found"){
                    if (ficha.link.text === value){
                        currentFiches[index].action = orig_ficha.action
                        currentFiches[index].action_value = orig_ficha.action_value
                        currentFiches[index].resolved = orig_ficha.resolved
                        updateFiche(index);
                    }
                }
            })
            break;
        case "macro_found":
            currentFiches.forEach((ficha, index) => {
                if (ficha.type === "macro_found"){
                    console.log("");
                    console.log(value);
                    console.log(ficha.macro.text);
                    console.log("");
                    if (ficha.macro.text === value){
                        currentFiches[index].action = orig_ficha.action
                        currentFiches[index].action_value = orig_ficha.action_value
                        currentFiches[index].resolved = orig_ficha.resolved
                        updateFiche(index);
                    }
                }
            })
            break;
    }
}
function resetAll() {
    loadFiches(originalData)
}
function submitFixes() {
    load_uid = originalData['load-uid']

    console.log(">>>>>>>>>>>>>>>")
    console.log(load_uid)

    docelmstyle('fichesBlock').display = 'none';
    docelmstyle('message').display = 'block';
    docelm('message').innerHTML = "Документы отправлены на индексацию";

    fetch('/send_files_to_index', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json',
        },
        body: JSON.stringify({ "load-uid": load_uid, "fiches": currentFiches })
    })
    .then(response => response.json())
    .then(data => {
        if (data.stats === 'error') {
            showError(data.message);
            return;
        }

        console.log("Ответ на загрузку")
        console.log(data)
        docelmstyle('fichesBlock').display = 'none';
        docelmstyle('message').display = 'block';
        docelm("message").innerHTML = data.message;

    })
    .catch(error => {
        showError('Ошибка при индексации: ' + error);
    });
}

Writing static/js/fileloader_fichesResolve.js


### Управление макросами

In [9]:
#@title macroses.html
%%writefile templates/macroses.html

<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <title>Управление макросами</title>
</head>
<body>
    <div style="margin-bottom: 20px;">
        <button onclick="goToMain()">На главную</button>
        <button onclick="saveMacros()">Сохранить</button>
    </div>

    <div id="macros-container">
        <!-- Макросы будут добавляться здесь -->
    </div>

    <script>
        // Загрузка макросов при открытии страницы
        window.addEventListener('DOMContentLoaded', loadMacros);

        function loadMacros() {
            fetch('/get_macros')
                .then(response => response.json())
                .then(macrosdb => {
                    const container = document.getElementById('macros-container');
                    container.innerHTML = '';

                    for (const [name, value] of Object.entries(macrosdb)) {
                        addMacroField(name, value);
                    }
                });
        }

        function addMacroField(name = '', value = '') {
            const container = document.getElementById('macros-container');
            const macroDiv = document.createElement('div');
            macroDiv.style.marginBottom = '15px';
            macroDiv.style.padding = '10px';
            macroDiv.style.border = '1px solid #ccc';

            macroDiv.innerHTML = `
                [macro@ <input type="text" value="${name}" placeholder="имя макроса" class="macro-name"> ]
                <br>
                <textarea placeholder="значение макроса" class="macro-value" rows="3" style="width: 300px; margin-top: 5px;">${value}</textarea>
                <br>
                <button onclick="this.parentElement.remove()">Удалить</button>
            `;

            container.appendChild(macroDiv);
        }

        // ОДИН обработчик для ВСЕХ полей ввода имен макросов
        document.addEventListener('input', function(event) {
            if (event.target.classList.contains('macro-name')) {
                event.target.value = event.target.value.replace(/\s+/g, '_');
            }
        });


        function addNewMacro() {
            addMacroField('', '');
        }

        function saveMacros() {
            const macros = {};
            const macroElements = document.querySelectorAll('#macros-container > div');

            macroElements.forEach(div => {
                const name = div.querySelector('.macro-name').value.trim();
                const value = div.querySelector('.macro-value').value.trim();

                if (name) {
                    macros[name] = value;
                }
            });

            fetch('/save_macros', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify(macros)
            })
            .then(response => response.json())
            .then(data => {
                if (data.status === 'success') {
                    alert('Макросы сохранены!');
                } else {
                    alert('Ошибка сохранения: ' + data.message);
                }
            });
        }

        function goToMain() {
            window.location.href = '/';
        }
    </script>

    <button onclick="addNewMacro()" style="margin-top: 20px;">+ Добавить макрос</button>
</body>
</html>

Writing templates/macroses.html


### Стратовая страница / Dashboard

In [10]:
 # @title index.html
%%writefile templates/index.html

<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Нейросотрудник</title>
</head>
<body>
    <div class="container">
        <div class="store-panel" style="margin: 20px 0; padding: 15px; border: 1px solid #ccc; background: #f9f9f9;">
            <h1>Vector Store</h1>
            <span id="store-status">
                {% if loading_docs %}
                    ⌛ Выполняется индексация документов
                {% elif is_store_inited %}
                    ✅ Хранилище готово к работе
                {% else %}
                    ❌ Не инициализирован
                {% endif %}
            </span>

            <!-- Статистика хранилища -->
            <div class="status-panel">
                <p>Документов: <span id="documents-count">{{ doc_count }}</span></p>
                <p>Нод: <span id="nodes-count">{{ node_count }}</span></p>
                <p>Макросов: <span id="macros-count">{{ macro_count }}</span></p>
                <form action="/macroses" style='display: {{ "block" if is_store_inited else "none" }}'>
                    <button id="to-macroses" type="submit"> Управление макросами </button>
                </form>
            </div>

            <!-- Панель управления -->
            <div class="control-panel">
                <form action="/initialize_storage">
                    <button id="initialize-btn" type="submit"> {{ "Загрузить пустое хранилище" if is_store_inited else "Инициализировать" }} </button>
                </form>

                <form id="import-form" enctype="multipart/form-data">
                    <input type="file" id="import-file" name="file" accept=".zip" style="display: none;">
                    <button type="button" onclick="document.getElementById('import-file').click()">Импорт хранилища</button>
                </form>

                <button id="export-btn" style='display: {{ "block" if is_store_inited else "none" }}'>Экспортировать</button>
                <form action="/load_documents">
                    <button id="load-docs-btn" type="submit" style='display: {{ "block" if is_store_inited else "none" }}' >&lt;&lt; Загрузить документы</button>
                </form>
            </div>
        </div>

        <!-- Панель трассировки -->
        <div class="tracing-panel" style="margin: 20px 0; padding: 15px; border: 1px solid #ccc; background: #f9f9f9;">
            <h3>Мониторинг LLM</h3>
            <div id="tracing-status">
                <p>Статус трассировки: <span id="tracing-status-text">Проверка...</span></p>
                <p id="phoenix-url" style="display: none;">URL: <span id="phoenix-url-text"></span></p>
            </div>
            <button id="phoenix-btn" style="display: none;" onclick="openPhoenix()">📊 Перейти к трассировке</button>
        </div>

        <!-- Поле для запросов к хранилищу -->
        <div class="query-panel" style='margin: 20px 0; padding: 10px; border: 1px solid #ccc; background: #f0fff8; display: {{ "block" if is_store_inited else "none" }}'>
            <h3 style="margin: 0; cursor: pointer;" onclick="togglePanel('query-panel-content', this)">🔍 Поиск в векторном хранилище</h3>
            <div id="query-panel-content" style="display: none; margin-top: 10px;">
                <textarea id="query-input" placeholder="Введите запрос для поиска в хранилище..." rows="3" style="width: 100%;"></textarea>
                <button onclick="queryVectorStore()">Выполнить запрос</button>
                <div id="query-results" style="margin-top: 20px; border: 1px solid #ccc; padding: 10px; min-height: 100px;">
                    <p>Результаты появятся здесь...</p>
                </div>
            </div>
        </div>

        <!-- Поле для запросов к LLM -->
        <div class="llm-chat-panel" style='margin: 20px 0; padding: 10px; border: 1px solid #ccc; background: #f0f8ff; display: {{ "block" if is_store_inited else "none" }}'>
            <h3 style="margin: 0; cursor: pointer;" onclick="togglePanel('llm-panel-content', this)">💬 Чат с LLM</h3>
            <div id="llm-panel-content" style="display: none; margin-top: 10px;">
                <textarea id="llm-input" placeholder="Введите ваш вопрос к языковой модели..." rows="4" style="width: 100%;"></textarea>
                <button onclick="sendToLLM()">Отправить запрос</button>
                <div id="llm-results" style="margin-top: 20px; border: 1px solid #ccc; padding: 10px; min-height: 100px; background: white;">
                    <p>Ответ LLM появится здесь...</p>
                </div>
            </div>
        </div>
    </div>

    <script>
        function togglePanel(panelId, header) {
            const panel = document.getElementById(panelId);
            if (panel.style.display === 'none') {
                panel.style.display = 'block';
            } else {
                panel.style.display = 'none';
            }
        }
    </script>

    <script>
        // Проверка статуса трассировки при загрузке страницы
        document.addEventListener('DOMContentLoaded', function() {
            checkTracingStatus();
        });

        // Функция проверки статуса трассировки
        function checkTracingStatus() {
            fetch('/status')
                .then(response => response.json())
                .then(data => {
                    const statusText = document.getElementById('tracing-status-text');
                    const phoenixBtn = document.getElementById('phoenix-btn');
                    const embeddedBtn = document.getElementById('phoenix-embedded-btn');
                    const phoenixUrl = document.getElementById('phoenix-url');
                    const phoenixUrlText = document.getElementById('phoenix-url-text');

                    if (data.services && data.services.phoenix_tracing) {
                        const phoenix = data.services.phoenix_tracing;

                        if (phoenix.status === 'running') {
                            statusText.textContent = '✅ Активна';
                            statusText.style.color = 'green';

                            // Показываем кнопки
                            phoenixBtn.style.display = 'inline-block';
                            phoenixUrl.style.display = 'block';

                            // Сохраняем URL для использования
                            phoenixUrlText.textContent = phoenix.public;
                            window.phoenixUrl = phoenix.public;
                        } else {
                            statusText.textContent = '❌ Неактивна';
                            statusText.style.color = 'red';
                        }
                    } else {
                        statusText.textContent = '❌ Недоступна';
                        statusText.style.color = 'red';
                    }
                })
                .catch(error => {
                    console.error('Ошибка проверки статуса трассировки:', error);
                    document.getElementById('tracing-status-text').textContent = '❌ Ошибка проверки';
                    document.getElementById('tracing-status-text').style.color = 'red';
                });
        }

        // Функция открытия Phoenix в новой вкладке
        function openPhoenix() {
            if (window.phoenixUrl) {
                window.open(window.phoenixUrl, '_blank');
            } else {
                // Если URL не сохранен, пытаемся получить его из элемента
                const url = document.getElementById('phoenix-url-text').textContent;
                if (url) {
                    window.open(url, '_blank');
                } else {
                    alert('URL трассировки не найден');
                }
            }
        }

        // Функция открытия встроенного Phoenix
        function openEmbeddedPhoenix() {
            window.open('/phoenix-embedded', '_blank');
        }

        // Кнопка экспорт
        document.getElementById('export-btn').addEventListener('click', function() {
            window.location.href = '/export_storage';
        });

        // Обработчик импорта
        document.getElementById('import-file').addEventListener('change', function() {
            if (this.files.length > 0) {
                const file = this.files[0];
                const formData = new FormData();

                // Старый импорт документов
                formData.append('file', file);

                fetch('/import_storage', {
                    method: 'POST',
                    body: formData
                })
                .then(response => response.json())
                .then(data => {
                    if (data.status === 'success') {
                        alert(data.message);
                        window.location.href = '/';
                    } else {
                        alert(data.message);
                    }
                })
                .catch(error => {
                    console.error('Error:', error);
                    alert('Ошибка при импорте файла');
                });

                // Сбрасываем input
                this.value = '';
            }
        });


        // Функция для запроса к векторному хранилищу
        function queryVectorStore() {
            let query = document.getElementById('query-input').value;
            let resultsDiv = document.getElementById('query-results');

            if (!query.trim()) {
                resultsDiv.innerHTML = '<p>Введите запрос</p>';
                return;
            }

            resultsDiv.innerHTML = '<p>Ищем...</p>';

            fetch('/query_vector_store', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({ query: query })
            })
            .then(response => response.json())
            .then(data => {
                if (data.status === 'success') {
                    let html = `<h4>Найдено нод: ${data.nodes.length}</h4>`;

                    data.nodes.forEach((node, index) => {
                        topics = ''
                        node.topics.forEach((topic) => {
                            topics = topics + `<strong>${topic}</strong><br>`
                        })

                        html += `
                            <div style="border: 1px solid #ddd; margin: 10px 0; padding: 10px;">
                                <strong>Нода ${index + 1}:</strong><br>
                                <strong>ID:</strong> ${node.id_}<br>
                                <strong>Файл:</strong> ${node.file_name}<br>
                                ${topics}
                                <strong>Текст:</strong> ${node.text}<br>
                                <strong>Сходство:</strong> ${node.similarity ? node.similarity.toFixed(3) : 'N/A'}
                            </div>
                        `;
                    });

                    resultsDiv.innerHTML = html;
                } else {
                    resultsDiv.innerHTML = `<p>Ошибка: ${data.message}</p>`;
                }
            })
            .catch(error => {
                console.error('Error:', error);
                resultsDiv.innerHTML = '<p>Ошибка при выполнении запроса</p>';
            });
        }

        // Функция для запроса к LLM
        function sendToLLM() {
            let query = document.getElementById('llm-input').value;
            let resultsDiv = document.getElementById('llm-results');

            if (!query.trim()) {
                resultsDiv.innerHTML = '<p>Введите запрос</p>';
                return;
            }

            resultsDiv.innerHTML = '<p>🤔 Обрабатываю запрос...</p>';

            fetch('/chat', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({ message: query })
            })
            .then(response => response.json())
            .then(data => {
                console.log(data)
                if (data.status === 'success') {
                    console.log("Получили ответ от LLM:")
                    console.log(data.response)
                    resultsDiv.innerHTML = `
                        <div style="border-left: 4px solid #007cba; padding-left: 10px;">
                            <strong>Ваш вопрос:</strong> ${query}<br><br>
                            <strong>Ответ LLM:</strong><br>
                            ${data.response.response.content.replace(/\n/g, '<br>')}
                        </div>
                    `;
                } else {
                    resultsDiv.innerHTML = `<p style="color: red;">Ошибка: ${data.message}</p>`;
                }
            })
            .catch(error => {
                console.error('Error:', error);
                resultsDiv.innerHTML = '<p style="color: red;">Ошибка при выполнении запроса к LLM</p>';
            });
        }
    </script>
</body>
</html>


Writing templates/index.html


### App

#### Вспомогательные классы

In [11]:
# @title MacroHandler

class MacroHandler:
    def __init__(self, functions_file, storageManager):
        self.functions = {}
        self.load_functions(functions_file)
        self.storageManager = storageManager

    def load_functions(self, file_path):
        """Динамически загружает функции из Python файла"""
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Functions file {file_path} not found")

        spec = importlib.util.spec_from_file_location("functions_module", file_path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)

        # Собираем все функции из модуля (исключая встроенные)
        for attr_name in dir(module):
            attr = getattr(module, attr_name)
            if callable(attr) and not attr_name.startswith('_'):
                self.functions[attr_name] = attr

    def execute_macro_function(self, function_name):
        """Выполняет макрос-функцию по имени"""
        if function_name in self.functions:
            return self.functions[function_name](self)
        else:
            return False

    def list_functions(self):
        """Возвращает список доступных функций"""
        return list(self.functions.keys())



    # Работа с ссылками и макросами в тексте #############

    def find_all_by_pattern(text, pattern, pattern_name):
        """
        Универсальная функция для поиска всех вхождений по regex паттерну.
        """
        matches = []

        try:
            # Компилируем паттерн если передан строкой
            if isinstance(pattern, str):
                compiled_pattern = re.compile(pattern, re.IGNORECASE)
            else:
                compiled_pattern = pattern

            # Ищем все совпадения
            for match in compiled_pattern.finditer(text):
                matched_text = match.group()
                start, end = match.span()

                # Извлекаем группы если есть
                groups = match.groups()
                named_groups = match.groupdict()

                matches.append({
                    'text': matched_text,
                    'type': pattern_name,
                    'start': start,
                    'end': end,
                    'groups': groups,
                    'named_groups': named_groups
                })

        except re.error as e:
            print(f"Ошибка в паттерне {pattern_name}: {e}")
        except Exception as e:
            print(f"Ошибка при поиске по паттерну {pattern_name}: {e}")

        return matches

    def normalize_link(link: str, link_type: str) -> str:
        """
        Умная нормализация ссылок.
        """
        original_link = link

        # Email - не меняем
        if link_type == 'email':
            return link.lower()

        # HTTP/HTTPS - уже нормализованы
        if link_type == 'http_https':
            return link.lower()

        # WWW домены - добавляем https://
        if link_type == 'www_domain' and not link.startswith(('http://', 'https://')):
            return 'https://' + link.lower()

        # Домены с путями
        if link_type == 'domain_with_path':
            if not link.startswith(('http://', 'https://')):
                return 'https://' + link.lower()
            return link.lower()

        # Простые домены (только если это действительно домен, а не часть слова)
        if link_type == 'domain_only':
            # Проверяем, что это отдельный домен, а не часть чего-то
            if '.' in link and not any(c in link for c in ['/', '\\', ' ']):
                return 'https://' + link.lower()
            return link

        # Файловые пути
        if link_type == 'file_path':
            # Нормализуем разделители
            link = link.replace('\\', '/')
            link = re.sub(r'/+', '/', link)
            return link

        # Внутренние ресурсы
        if link_type == 'internal_resource':
            # Убираем лишние слеши
            link = re.sub(r'/+', '/', link)
            return link

        return original_link

    def find_all_links(text):
        """
        Находит все ссылки в тексте, избегая дубликатов и перекрывающихся вхождений.
        """
        # паттерны с приоритетами
        patterns = [
            # Высокий приоритет - полные URL
            ('http_https', r'https?://[^\s<>"{}|\\^`\[\]]+'),
            ('email', r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'),
            ('www_domain', r'www\.[^\s<>"{}|\\^`\[\]]+\.[a-zA-Z]{2,}(?:/[^\s<>"{}|\\^`\[\]]*)?'),

            # Средний приоритет - домены и файловые пути
            ('file_path', r'(?:[a-zA-Z]:\\[^\s<>"|]+?\.\w{2,4})|/(?:[^\s<>"]+/)+[^\s<>"]+\.\w{2,4}'),
            ('domain_with_path', r'[a-zA-Z0-9-]+\.[a-zA-Z]{2,}(?::\d+)?(?:/[^\s<>"{}|\\^`\[\]]+)'),

            # Низкий приоритет - простые домены
            ('domain_only', r'\b[a-zA-Z0-9-]+\.[a-zA-Z]{2,}(?::\d+)?\b'),

            # Внутренние ресурсы; если содержат разделители пути
            ('internal_resource', r'/(?:[^\s<>"/]+/)+[^\s<>"/]+(?:\.[a-zA-Z0-9]{2,4})?'),  # Unix paths
            ('internal_resource_win', r'[a-zA-Z]:\\(?:[^\s<>"\\]+\\)+[^\s<>"\\]+(?:\.[a-zA-Z0-9]{2,4})?'),  # Windows paths

            # URL-параметры и якоря
            ('url_params', r'/[^\s<>"]+\?[^\s<>"]+'),  # С параметрами
            ('url_anchor', r'/[^\s<>"]+#[^\s<>"]+'),    # С якорями
        ]

        all_matches = []
        covered_positions = set()

        # Ищем по всем паттернам в порядке приоритета
        for pattern_name, pattern in patterns:
            matches = MacroHandler.find_all_by_pattern(text, pattern, pattern_name)

            for match in matches:
                start, end = match['start'], match['end']

                # Проверяем, не перекрывается ли с уже найденными
                is_overlap = any(start < existing_end and end > existing_start
                              for existing_start, existing_end in covered_positions)

                if not is_overlap:
                    # Нормализуем ссылку
                    match['normalized'] = MacroHandler.normalize_link(match['text'], pattern_name)
                    all_matches.append(match)

                    # Добавляем позицию в покрытые
                    covered_positions.add((start, end))

        # Сортируем по позиции
        all_matches.sort(key=lambda x: x['start'])

        return all_matches

    def find_all_macro(text, macro_tag = 'macro'):
        """
        Находит все макросы в тексте вида [macro@имя].
        """
        # Стандартный паттерн для макросов [macro@имя]
        macro_pattern = fr'\[{macro_tag}@([^\]]+)\]'
        macros = MacroHandler.find_all_by_pattern(text, macro_pattern, macro_tag)

        # Добавляем информацию о имени макроса
        for macro in macros:
            if macro['groups']:
                macro['macro_name'] = macro['groups'][0]  # Первая группа - имя макроса
            else:
                # Если нет групп, извлекаем имя из текста
                macro_text = macro['text']
                if macro_text.startswith('[macro@') and macro_text.endswith(']'):
                    macro['macro_name'] = macro_text[7:-1]  # Убираем [macro@ и ]
                else:
                    macro['macro_name'] = macro_text

        return macros


    def execute_all_macroses_in_text(self,
                                     text,
                                     level = 0,
                                     no_value_filler = '[Сообщение от @RAG для @Agent: тут должно быть заначение макроса, но оно не было найдено.]'):
        """
        Находит все макросы в тексте и выполняет их, заменяя текст макроса результатом его выполнения.
        Операция рекурсивно выполняется для результата выполененеия макроса *(глубина рекурсии до 10)
        Возвращает обновленный text.
        """
        level = level + 1
        if level > 10:
            return

        macroses = MacroHandler.find_all_macro(text, 'macro')
        macro_code = MacroHandler.find_all_macro(text, 'macro_func')

        # объединяем и сортируем по 'start'
        combined = macroses + macro_code
        sorted_combined = sorted(
            combined,
            key=lambda item: item['start']
        )

        # {'text': '[macro@тестовая_ссылка_1]', 'type': 'macro', 'start': 121, 'end': 146, 'groups': ('тестовая_ссылка_1',), 'named_groups': {}, 'macro_name': 'тестовая_ссылка_1'}

        for i, macro in enumerate(sorted_combined):
            # print(f"-] Обрабатываю {macro}")

            macro_name = macro['macro_name']
            result_txt = no_value_filler

            match macro["type"]:
                case 'macro':
                    # print(f"-]-] Это macro")
                    macro_value = self.storageManager.get_macro_value(macro['macro_name'])
                    result_txt = macro_value or result_txt

                case 'macro_func':
                    # print(f"-]-] Это macro_func")
                    if macro_name in self.list_functions():
                        function_result = self.execute_macro_function(macro_name)
                        result_txt = function_result or result_txt
            # print(f'-]-]-] result: {result_txt}')

            result_txt = self.execute_all_macroses_in_text(result_txt, level, no_value_filler) or result_txt

            # Заменяем оригинальный текст макроса на результат его выполнения
            text = text[:macro['start']] + result_txt + text[macro['end']:]
            delta = len(result_txt) - (macro['end'] - macro['start'])

            # Смещаем остальные макросы
            for j, subc_macro in enumerate(sorted_combined):
                if j > i:
                    subc_macro['start'] += delta
                    subc_macro['end'] += delta
        return text



In [27]:
# @title StorageManager

class StorageManager:
    def __init__(self):
        self.index = None
        self.macros_db = None
        self.query_engine = None
        self.is_store_inited = False
        self.is_query_inited = False
        self.uploading_docs = False
        self.macroHandler = None

        self.load_macro_handler("/content/macro_functions.py")

        self.documents = []
        self.node_count = 0
        self.macro_count = 0

        self.loader_temp_data_storage = {}

        self.persist_dir = "./storage"
        self.macros_file = "./storage/macros.json"

    def load_macro_handler(self, filepath):
        self.macroHandler = MacroHandler(filepath, self)

    def init_query_engine(self):
        # self.colbert_reranker = ColbertRerank(
        #     top_n=15,
        #     model="colbert-ir/colbertv2.0",
        #     tokenizer="colbert-ir/colbertv2.0",
        #     keep_retrieval_score=True,
        # )
        # self.query_engine = self.index.as_query_engine(
        #     similarity_top_k=20,
        #     similarity_cutoff=0.1,
        #     vector_store_query_mode="hybrid",
        # )
        self.query_engine = self.index.as_query_engine(
            similarity_top_k=15,
            similarity_cutoff=0.15,
            vector_store_query_mode="default",
            embed_model=Settings.embed_model,
            # node_postprocessors=[self.colbert_reranker]
        )

    def get_query_engine(self):
        if not self.is_store_inited:
            return False

        if not self.query_engine:
          self.init_query_engine()

        return self.query_engine

    def init_clear_store(self):
        if self.uploading_docs:
            return

        self.documents = []
        self.node_count = 0
        self.macro_count = 0
        self.loader_temp_data_storage = {}

        self.index = VectorStoreIndex(
            nodes=[],
            embed_model=Settings.embed_model,
            llm=Settings.llm
        )
        self.init_query_engine()

        self.macros_db = {}

        self.is_store_inited = True

    def retrieve_nodes(self, query_str):
        query_bundle = QueryBundle(query_str=query_str)
        query_engine = self.get_query_engine()
        retrieved_nodes = query_engine.retrieve(query_bundle)
        if not retrieved_nodes:
            return {"retrieved_nodes": ["Информации по данному запросу в хранилище не найдено"]}

        try:
            context_text = ""
            for node in retrieved_nodes:
                filename = node.metadata.get('file_name', 'Документ без имени')
                topics = ""
                tpc = 1
                while tpc != 0:
                    tpc += 1
                    topic = node.metadata.get(f'topic_level_{tpc}', False)
                    if not topic:
                        tpc = 0
                    else:
                        topics += "#"*tpc + topic + '\n'
                context_text = context_text + f'\n#{filename}\n{topics}\n{node.text}\n'

        except Exception as e:
            error_traceback = traceback.format_exc()
            print(f"❌) StorageManager error при формировании контекста: \n{error_traceback}\n❌")

        return {"context_text": context_text, "retrieved_nodes": retrieved_nodes}

    def get_chashed_info(self):
        return {
            "is_store_inited": self.is_store_inited,
            "doc_count": len(self.documents),
            "node_count": self.node_count,
            "macro_count": self.macro_count,
            "loading_docs": self.uploading_docs,
        }

    # Скинуть данные хранилища в файлы
    def save_storage(self):
        """Сохраняет индекс и данные в файлы"""
        import os
        os.makedirs(self.persist_dir, exist_ok=True)

        # Сохраняем индекс через встроенную persistence
        if self.index:
            self.index.storage_context.persist(persist_dir=self.persist_dir)

        # Сохраняем макросы в JSON
        with open(self.macros_file, 'w', encoding='utf-8') as f:
            json.dump(self.macros_db, f, ensure_ascii=False, indent=2)

        # Сохраняем метаданные
        metadata = {
            'documents': list(self.documents),
            'node_count': self.node_count,
            'macro_count': self.macro_count,
        }
        with open(f"{self.persist_dir}/metadata.json", 'w') as f:
            json.dump(metadata, f)

    def export_storage_archive(self):
        """Создает zip-архив со всеми данными"""

        # Сначала сохраняем всё
        self.save_storage()

        # Создаем архив
        zip_buffer = io.BytesIO()
        with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
            # Добавляем все файлы из папки storage
            for root, dirs, files in os.walk(self.persist_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, self.persist_dir)
                    zip_file.write(file_path, arcname)

            # Добавляем файл макросов
            if os.path.exists(self.macros_file):
                zip_file.write(self.macros_file, "macros.json")

        zip_buffer.seek(0)
        return zip_buffer

    def load_storage(self):
        if self.uploading_docs:
            return

        """Загружает индекс и данные из файлов"""

        if not os.path.exists(self.persist_dir):
            return False

        try:
            # Загружаем индекс
            storage_context = StorageContext.from_defaults(persist_dir=self.persist_dir)
            self.index = load_index_from_storage(storage_context)

            # Загружаем макросы
            if os.path.exists(self.macros_file):
                with open(self.macros_file, 'r', encoding='utf-8') as f:
                    self.macros_db = json.load(f)

            # Загружаем метаданные
            if os.path.exists(f"{self.persist_dir}/metadata.json"):
                with open(f"{self.persist_dir}/metadata.json", 'r') as f:
                    metadata = json.load(f)
                    self.documents = metadata['documents']
                    self.node_count = metadata['node_count']
                    self.macro_count = metadata['macro_count']

            self.init_query_engine()
            self.is_store_inited = True
            return True

        except Exception as e:
            print(f"Ошибка загрузки: {e}")
            return False

    def import_storage_archive(self, zip_data):
        if self.uploading_docs:
            return

        """Импортирует из zip-архива"""
        import zipfile
        import shutil
        import os

        # Очищаем текущую папку
        if os.path.exists(self.persist_dir):
            shutil.rmtree(self.persist_dir)

        # Распаковываем архив
        with zipfile.ZipFile(io.BytesIO(zip_data), 'r') as zip_file:
            zip_file.extractall(self.persist_dir)

        # Загружаем данные
        return self.load_storage()


    def get_documents_list(self):
        return self.documents

    def get_macro_db(self):
        return self.macros_db

    def set_macro_db(self, macro_db):
        self.macros_db = macro_db
        self.macro_count = len(macro_db.items())

    def add_macro(self, macroname, macrovalue):
        self.macros_db[macroname] = macrovalue
        self.macro_count+=1

    def del_macro(self, macroname):
        del self.macros_db[macroname]
        self.macro_count-=1

    def get_macro_value(self, macroname):
        return self.macros_db.get(macroname, False)

    def extract_macro_name(self, macro_str):
        pattern = r'@([^\]]+)'
        match = re.search(pattern, macro_str)
        if match:
            result_regex = match.group(1)
            return result_regex

    def get_macro_by_value(self, value):
        return next((key for key, val in self.macros_db.items() if val == value), False)


    def upload_nodes(self, nodes, documents_list):
        self.uploading_docs = True
        for node in nodes:
            self.index.insert_nodes([node])
            print(f"--> загружена нода {node.id_}")

        for item in documents_list:
            print(f"обработанный документ {item}")
            if item not in self.documents:
                self.documents.append(item)
                print(f"   добавлен")
        self.node_count += len(nodes)
        self.uploading_docs = False


    def save_temp_loader_data(self, data):
        tstamp = str(int(time.time() * 1000))
        self.loader_temp_data_storage[tstamp] = data
        return tstamp

    def load_temp_loader_data(self, uid):
        return self.loader_temp_data_storage[uid]

In [13]:
#@title VseGPT_LLM

class VseGPT_LLM(CustomLLM):
    """
    Кастомная LLM для работы с VseGPT API.
    """

    model_name: str
    temperature: float = 0.1
    context_window: int
    max_tokens: int = 2048
    api_url: str = "https://api.vsegpt.ru/v1/chat/completions"
    api_key: str
    system_prompt: str = "Ты бот, отвечай кратко и точно."

    @property
    def metadata(self) -> LLMMetadata:
        """Возвращает метаданные LLM."""
        return LLMMetadata(
            context_window=self.context_window,
            num_output=1024,
            model_name=self.model_name,
        )

    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        """Основной метод для выполнения запроса к API."""

        # заголовки запроса как требует VseGPT
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }

        # Добавляем системный промпт если он есть
        messages = []
        if self.system_prompt:
            messages.append({"role": "system", "content": self.system_prompt})
        messages.append({"role": "user", "content": prompt})

        # тело запроса в формате VseGPT/OpenAI API
        payload = {
            "model": self.model_name,
            "messages": messages,
            "temperature": self.temperature,
            "max_tokens": self.max_tokens,
            "stream": False
        }

        print(f"\n>>>>>>>>>>>>>>>>>>>>>>>>>VseGPT_LLM PAYLOAD SEND ------>>>>>>")
        print(payload)
        print("-----")

        try:
            # POST-запрос к API
            response = requests.post(self.api_url, json=payload, headers=headers)
            response.raise_for_status()  # Бросит исключение для HTTP ошибок (4xx, 5xx)

            # JSON ответ
            response_data = response.json()
            completed_text = response_data['choices'][0]['message']['content'].strip()
            print(f"---<<< {self.model_name} response")
            print(completed_text)
            print(">>>>>>>>>>>>>>>>>>>>>>>>>\n")

            return CompletionResponse(text=completed_text)

        except requests.exceptions.RequestException as e:
            # Обработка ошибок сети/HTTP
            raise Exception(f"Ошибка запроса к VseGPT API: {str(e)}")
        except KeyError as e:
            # Обработка ошибок парсинга JSON (неожиданная структура ответа)
            raise Exception(f"Не удалось распарсить ответ от API. Структура ответа: {response_data}")

    # Нужен для стриминга, но он опционален. Просто заглушим.
    @llm_completion_callback()
    def stream_complete(self, prompt: str, **kwargs: Any) -> CompletionResponseGen:
        yield CompletionResponse(text="Стриминг не реализован для этого провайдера.")

In [14]:
#@title HierarchicalSmartChunker

class HierarchicalSmartChunker(SentenceSplitter):
    """
    Чанкер, который парсит иерархические заголовки (##, ###, ####)
    и добавляет их в метаданные каждого чанка.
    """

    header_pattern: str = r"^(#+)\s*(.+?)\s*$"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def _parse_nodes(self, nodes: List[TextNode], **kwargs) -> List[TextNode]:
        all_new_nodes = []

        for node in nodes:
            text = node.get_content()
            lines = text.split('\n')  # Разбиваем текст на строки

            # Инициализируем состояние для этого документа
            current_headers: Dict[int, str] = {}  # Уровень -> текст заголовка
            buffer = []  # Буфер для накопления строк текущего чанка
            new_nodes = []  # Список чанков для этого документа

            def _flush_buffer_to_node():
                """Вспомогательная функция: создает ноду из буфера и сбрасывает его."""
                nonlocal buffer, current_headers
                if buffer:
                    chunk_text = '\n'.join(buffer).strip()
                    if chunk_text:  # Создаем чанк только если есть текст
                        metadata = node.metadata.copy()

                        # Добавляем все актуальные заголовки в метаданные
                        for level, header_text in current_headers.items():
                            metadata[f'topic_level_{level}'] = header_text

                        new_node = TextNode(
                            text=chunk_text,
                            metadata=metadata,
                        )
                        new_nodes.append(new_node)
                    buffer = []

            # Основной цикл обработки строк
            for line in lines:
                line = line.strip()
                if not line:
                    continue

                # Проверяем, является ли строка заголовком
                header_match = re.match(self.header_pattern, line)
                if header_match:
                    # Нашли заголовок - завершаем текущий чанк
                    _flush_buffer_to_node()

                    # Определяем уровень и текст заголовка
                    hashes, header_text = header_match.groups()
                    level = len(hashes)  # Уровень вложенности (2, 3, 4...)

                    # Обновляем состояние заголовков
                    # Удаляем все заголовки более глубоких уровней
                    keys_to_remove = [l for l in current_headers.keys() if l >= level]
                    for key in keys_to_remove:
                        current_headers.pop(key, None)

                    # Добавляем текущий заголовок
                    current_headers[level] = header_text

                    # Не добавляем сам заголовок в буфер - он уже учтен в метаданных
                    continue

                # Если это не заголовок, добавляем строку в буфер
                buffer.append(line)

            # После обработки всех строк - сохраняем последний чанк
            _flush_buffer_to_node()
            all_new_nodes.extend(new_nodes)

        return all_new_nodes

#### Функции приложения

In [15]:
#@title Вспомогательные функции

def add_element_to_array(d, key, value):
    d.setdefault(key, []).append(value)

##### Работа файлами и индексация

In [16]:
#@title Парсинг нод, обработка замечаний


def scan_directory(path, parent_folder_path=None):
    """Рекурсивно сканирует директорию и возвращает древовидную структуру"""
    if not os.path.exists(path):
        return None

    result = {
        'name': os.path.basename(path),
        'path': path,
        'type': 'folder',
        'children': [],
        'expanded': False,
        'selected': False,
        'parent_path': parent_folder_path
    }

    try:
        items = os.listdir(path)

        # Разделяем на папки и файлы
        folders = []
        files = []

        for item in items:
            item_path = os.path.join(path, item)
            if os.path.isdir(item_path):
                folders.append((item, item_path))
            else:
                if item.lower().endswith(('.txt', '.pdf', '.docx', '.doc', '.md')):
                    files.append((item, item_path))

        # Сортируем папки по алфавиту и обрабатываем
        for item, item_path in sorted(folders, key=lambda x: x[0].lower()):
            child_folder = scan_directory(item_path, path)
            if child_folder:
                result['children'].append(child_folder)

        # Сортируем файлы по алфавиту и добавляем
        for item, item_path in sorted(files, key=lambda x: x[0].lower()):
            result['children'].append({
                'name': item,
                'path': item_path,
                'type': 'file',
                'selected': False,
                'parent_path': path
            })

    except PermissionError:
        pass

    return result


def get_selected_files(tree, selected_files):
    """Рекурсивно собирает все выбранные файлы из дерева"""
    if tree['type'] == 'file' and tree.get('selected'):
        selected_files.append(tree['path'])

    for child in tree.get('children', []):
        get_selected_files(child, selected_files)


def collect_preprocess_files_fiches(files, storageManager):
    """ Собирает все фичи для загружаемых файлов и варианты их устранения """
    storage_files = storageManager.get_documents_list()

    # Собираем все тексты
    documents = SimpleDirectoryReader(input_files=files).load_data()

    # Режем на чанки
    chunker = HierarchicalSmartChunker(
        chunk_size=4000,
        chunk_overlap=0,
        separator=" "
    )

    processed_nodes = chunker.get_nodes_from_documents(documents)

    # print()
    # print(f"Все файлы: {files}")
    # print(f"Все ноды: {processed_nodes}")
    files_already_in_storage = {}
    total_fiches = []

    # print("\n\n")
    for i, node in enumerate(processed_nodes):
        file_name = node.metadata['file_name']

        if file_name in storage_files:
            files_already_in_storage[file_name] = True

        topics = {}
        topic_level = 1
        while True:
            topic_level+=1
            topic = f"topic_level_{topic_level}"
            if not topic in node.metadata:
                break
            else:
                topics[str(topic_level)] = node.metadata[topic]

        # print(f"---\nФайл: {file_name}")
        # for tpc_num, topic in topics.items():
        #     print(f"Заголовок {tpc_num}: {topic}")
        # print(f' -=- \n{node.text}')
        # print()

        links = MacroHandler.find_all_links(node.text)
        for link in links:
            total_fiches.append({
                "type": "link_found",
                "node": node,
                "link": link,
                "file_name": file_name,
            })
        macroses = MacroHandler.find_all_macro(node.text)
        for macro in macroses:
            total_fiches.append({
                "type": "macro_found",
                "node": node,
                "macro": macro,
                "file_name": file_name,
            })

    for file_in_storage in reversed(files_already_in_storage.keys()):
        total_fiches.insert(0,{
            "type": "document_in_index",
            "file_name": file_in_storage,
        })

    print("\n\n")

    # print("Найденные фичи:")
    # for ficha in total_fiches:
    #     print()
    #     print(ficha["type"])

    #     if ficha["type"] == 'document_in_index':
    #         print(ficha["file_name"])

    #     if ficha["type"] == 'link_found':
    #         print(ficha["link"]['text'])
    #         print(' -=- ')
    #         print(ficha["node"].text)

    #     if ficha["type"] == 'macro_found':
    #         print(ficha["macro"]['text'])
    #         print(' -=- ')
    #         print(ficha["node"].text)


    return processed_nodes, total_fiches

In [17]:
#@title Подготовка замечаний к отправке клиенту

def prepare_fiches_to_send(fiches, storageManager):
    node_texts = {}
    prepared = []
    for ficha in fiches:
        collected = {
            "action": "",
            "action_value": "",
            "type": ficha["type"],
            "file_name": ficha["file_name"]}

        if (ficha["type"] == 'link_found') or (ficha["type"] == 'macro_found'):
            if ficha["type"] == 'link_found':
                collected["link"] = ficha["link"]
                exsit_macro = storageManager.get_macro_by_value(collected["link"]["text"])
                if exsit_macro:
                    collected["action_value"] = exsit_macro
                    collected["link"]["existing_macro"] = exsit_macro


            if ficha["type"] == 'macro_found':
                collected["macro"] = ficha["macro"]
                macroname = storageManager.extract_macro_name(collected["macro"]["text"])
                collected["macro"]["existing_macro"] = storageManager.get_macro_value(macroname)
                collected["macro"]["macro_name"] = macroname

            node = ficha["node"]

            topics = {}
            topic_level = 1
            while True:
                topic_level+=1
                topic = f"topic_level_{topic_level}"
                if not topic in node.metadata:
                    break
                else:
                    topics[str(topic_level)] = node.metadata[topic]

            collected["node"] = {
                "file_name": node.metadata['file_name'],
                "topics": topics,
                "id_": node.id_
            }
            node_texts[node.id_] = node.text

        elif ficha["type"] == 'document_in_index':
            collected["file_name"] = ficha["file_name"]

        prepared.append(collected)

    print(node_texts)
    return prepared, node_texts

In [18]:
#@title Обработка замечаний

def handle_fiches(nodes, fiches, storageManager):
    replace_value_nodes = {}
    do_not_index_files = []
    link_replace_macroses = []

    # Убираем не идексируемые документы
    for ficha in fiches:
        if ficha["type"] == "document_in_index":
            if ficha['action'] == 'cancel':
                do_not_index_files.append(ficha['file_name'])

    # Замена текста
    ## Соберем все замены для каждой из нод
    for ficha in fiches:
        if ficha['file_name'] in do_not_index_files:
            continue

        match ficha["type"]:
            case "link_found":
                replace_value = False
                if ficha["action"] == 'replace_macro':
                    replace_value = f"[macro@{ ficha["action_value"] }]"
                    replace_value = replace_value.replace(" ", "_")
                    link_replace_macroses.append({"macroname": ficha["action_value"], "macrovalue": ficha["link"]["text"]})

                if ficha["action"] == 'replace_text':
                    replace_value = ficha["action_value"]

                if replace_value:
                    id = ficha["node"]["id_"]
                    add_element_to_array(replace_value_nodes, id, {
                        "replace": ficha["link"]["text"],
                        "value": replace_value,
                        "start": ficha["link"]["start"]
                    })

            case "macro_found":
                replace_value = False
                if ficha["action"] == 'rename':
                    replace_value = f"[macro@{ ficha["action_value"] }]"
                    replace_value = replace_value.replace(" ", "_")
                    if not storageManager.get_macro_value(ficha["action_value"]):
                        link_replace_macroses.append({"macroname": ficha["action_value"], "macrovalue": "[-ЗНАЧЕНИЕ МАКРОСА НЕ ЗАДАНО-]"})

                if ficha["action"] == 'keep':
                    macroname = storageManager.extract_macro_name(ficha['macro']['text'])
                    link_replace_macroses.append({"macroname": macroname, "macrovalue": "[-ЗНАЧЕНИЕ МАКРОСА НЕ ЗАДАНО-]"})

                if ficha["action"] == 'replace_text':
                    replace_value = ficha["action_value"]

                if replace_value:
                    id = ficha["node"]["id_"]
                    add_element_to_array(replace_value_nodes, id, {
                        "replace": ficha["macro"]["text"],
                        "value": replace_value,
                        "start": ficha["macro"]["start"]
                    })

            case "document_in_index":
                pass

    print(replace_value_nodes)

    # Ищем новые макросы (которых нет в macro_db) и добавляем их в базу
    for macroreplace in link_replace_macroses:
        if not storageManager.get_macro_value(macroreplace['macroname']):
            storageManager.add_macro(macroreplace['macroname'], macroreplace['macrovalue'])

    processed_documents = set()

    # Применяем замены
    updated_nodes = []
    for node in nodes:
        node_id = node.id_
        document = node.metadata.get('file_name')
        if document in do_not_index_files:
            continue

        processed_documents.add(document)

        if node_id in replace_value_nodes:
            current_text = node.text
            replacements = replace_value_nodes[node_id]


            # заменяем поочередно все вхождения
            for replace_index, replacement in enumerate(replacements):
                old_text = replacement["replace"]
                new_text = replacement["value"]
                delta_start = len(new_text) - len(old_text)

                # Разделяем строку на части
                prefix = current_text[:replacement["start"]]
                suffix = current_text[replacement["start"]:]

                # Заменяем и собираем обратно
                current_text = prefix + suffix.replace(old_text, new_text, 1)

                #Смещаем идексы остальных замен
                for replace_index_2, replacement_2 in enumerate(replacements):
                    if replace_index_2 > replace_index:
                        replacement_2["start"] += delta_start

            # Создаем новую TextNode
            new_node = TextNode(
                text=current_text,
                metadata=node.metadata.copy(),
                id_=node.id_
            )
            updated_nodes.append(new_node)
        else:
            updated_nodes.append(node)

    return updated_nodes, list(processed_documents)

#### Guardrails

In [19]:
#@title VseGPT_LLM_Wrapper

from langchain.schema import LLMResult, Generation
from langchain.callbacks.manager import AsyncCallbackManagerForLLMRun, CallbackManagerForLLMRun
from typing import Any, List, Optional, Dict, Union

class VseGPT_LLM_Wrapper:
    """Полнофункциональный wrapper с поддержкой LangChain интерфейса"""

    def __init__(self, llm_instance):
        self.llm = llm_instance
        self.temperature = getattr(llm_instance, 'temperature', 0.1)
        self.max_tokens = getattr(llm_instance, 'max_tokens', 1024)
        self.model_name = getattr(llm_instance, 'model_name', 'vsegpt')

    # 🔧 ОСНОВНЫЕ СИНХРОННЫЕ МЕТОДЫ
    def invoke(self, prompt: Union[str, List[Dict]], **kwargs) -> str:
        """Синхронный вызов"""
        if isinstance(prompt, list):
            prompt = self._messages_to_prompt(prompt)
        response = self.llm.complete(prompt)
        return response.text

    def generate(self, prompts: List[str], **kwargs) -> LLMResult:
        """Синхронная генерация"""
        generations = []
        for prompt in prompts:
            response = self.llm.complete(prompt)
            generations.append([Generation(text=response.text)])
        return LLMResult(generations=generations)

    # 🔧 ОСНОВНЫЕ АСИНХРОННЫЕ МЕТОДЫ
    async def ainvoke(self, prompt: Union[str, List[Dict]], **kwargs) -> str:
        """Асинхронный вызов - ОБЯЗАТЕЛЬНЫЙ ДЛЯ GUARDRAILS"""
        return self.invoke(prompt, **kwargs)

    async def agenerate(self, prompts: List[str], **kwargs) -> LLMResult:
        """Асинхронная генерация"""
        return self.generate(prompts, **kwargs)

    async def _acall(self, prompt: str, **kwargs) -> str:
        """Альтернативный асинхронный вызов"""
        return await self.ainvoke(prompt, **kwargs)

    # 🔧 МЕТОДЫ ДЛЯ СОВМЕСТИМОСТИ
    def bind(self, **kwargs):
        """Для совместимости с цепочками"""
        return self

    def with_config(self, **kwargs):
        """Для конфигурации"""
        return self

    @property
    def input_keys(self) -> List[str]:
        return ["input"]

    @property
    def output_keys(self) -> List[str]:
        return ["output"]

    # 🔧 ВСПОМОГАТЕЛЬНЫЕ МЕТОДЫ
    def _messages_to_prompt(self, messages: List[Dict]) -> str:
        """Конвертирует сообщения в промпт"""
        prompt = ""
        for msg in messages:
            role = msg.get("role", "")
            content = msg.get("content", "")
            if role == "system":
                prompt += f"System: {content}\n\n"
            elif role == "user":
                prompt += f"User: {content}\n\n"
            elif role == "assistant":
                prompt += f"Assistant: {content}\n\n"
        return prompt.strip()

# Регистрируем провайдеры
register_llm_provider("vsegpt", VseGPT_LLM_Wrapper)

In [20]:
#@title GuardrailsManager

class GuardrailsManager:
    def __init__(self, storageManager, tool_llm, config_path: str = None):
        if config_path is None:
            config_path = '/content/guardrails'

        self.config = RailsConfig.from_path(str(config_path))
        self.rails = LLMRails(config=self.config, llm=VseGPT_LLM_Wrapper(tool_llm))
        self.storageManager = storageManager
        self.tool_llm = tool_llm

        # Регистрируем кастомные экшены
        self.rails.register_action(self.generate_rag_answer, "generate_rag_answer")
        print("✅ GuardrailsManager инициализирован!")

    @action()
    async def generate_rag_answer(self, context: dict = None):
        """Найти информацию в базе и дать ответ"""

        user_message = context.get('user_message') or context.get('last_user_message')
        print("\n\n---<RAG> ЗАПУСТИЛИ RAG ---")
        print(context)

        try:
            if not storageManager.is_store_inited:
                return "❌ База знаний недоступна"

            print("--запросили контекст---")
            context = storageManager.retrieve_nodes(user_message)
            print(f"--найдено нод: {len(context['retrieved_nodes'])} --")
            context_text = context['context_text']

            # Запускаем все макросы и заменяем их результатами выполнения
            context_text = self.storageManager.macroHandler.execute_all_macroses_in_text(context_text)

            # 🔥 Используем качественную модель напрямую
            # llm уже задана через Settings
            # Формируем ответ RAG
            prompt = f"""Инструкция для @Agent: Отвечай на вопросы на основе предоставленных документов и инструкций. Не все документы и инструкции относятся к теме вопроса.
Обрати внимание на загловки и имена документов, они помогают найти разделы с ответом (начинаются с нескольких знаков #). Если информации нет или недостаточно, то скажи об этом прямо.

Документы и инструкции:
{context_text}

Вопрос: {user_message}

Ответ:"""
            print(f"---> отправляем запрос к модели \n-------'''{prompt}'''-------\n</RAG>")
            response = Settings.llm.complete(prompt)
            print("\n<RAG> ответ RAG:--------\n'''")
            print(str(response))
            print("'''\n</RAG>---------\n")
            return str(response)

        except Exception as e:
            error_traceback = traceback.format_exc()
            print(f"❌❌❌ Guardrail RAG error: \n{error_traceback}\n❌❌❌")
            return f"❌ Ошибка: {str(e)}"


    async def process_message(self, user_message: str):
        """Основной метод обработки сообщения"""
        try:
            # Обрабатываем через Guardrails
            result = await self.rails.generate_async(
                messages=[{"role": "user", "content": user_message}]
            )

            return {
                "success": True,
                "response": result,
                "safety_checked": True
            }

        except Exception as e:
            print(f"\n\nОшибка в process_message {self.tool_llm.model_name}: {e}")
            return {
                "success": False,
                "error": str(e),
                "response": "Произошла ошибка при обработке запроса."
            }

In [21]:
#@title guardrails/config.yaml
%%writefile guardrails/config.yaml

models:
  # быстрая модель для проверок (tool_model)
  - type: main
    engine: vsegpt-tool
    model: vsegpt-tool

instructions:
  - type: general
    content: |
      Ты - робот классификатор. Задача: определить ТОЛЬКО intent из списка.

      Доступные интенты:
      - greeting (приветствия)
      - express goodbye (прощания)
      - ask about capabilities (о возможностях системы)
      - any question (любые вопросы)

      Правила классификации:
      - "расскажи о себе", "что ты умеешь" → ask about capabilities
      - "расскажи про X", "кто такой X", "что такое X" → any question
      - "привет", "здравствуйте" → greeting
      - "пока", "до свидания" → express goodbye

      1. НИКОГДА не придумывай ответы или объяснения
      2. НИКОГДА не продолжай разговор
      3. Верни ТОЛЬКО название интента

      Запомни: You are NOT an assistant, you are a CLASSIFIER.

rails:
  input:
    flows:
      - self check input
  output:
    flows:
      - self check output
  config:
    flows:
      - unsafe_input
      - rag_flow

prompts:
  - task: self_check_input
    content: |
      Проверь, содержит ли пользовательское сообщение небезопасный контент согласно нашей политике безопасности.
      Категории небезопасного контента:
      S1: Насилие
      S2: Сексуальный контент
      S3: Планирование преступлений/Признания
      S4: Оружие и незаконное оружие
      S5: Контролируемые/Регулируемые вещества
      S6: Суицид и самоповреждение
      S7: Сексуальный контент (несовершеннолетние)
      S8: Ненависть/Идентификационная ненависть
      S9: PII/Конфиденциальность
      S10: Харассмент
      S11: Угрозы
      S12: Ненормативная лексика
      S13: Требует осторожности
      S14: Другое
      S15: Манипуляции
      S16: Мошенничество/Обман
      S17: Вредоносное ПО
      S18: Высокорисковые государственные решения
      S19: Политика/Дезинформация/Теории заговора
      S20: Авторские права/Товарные знаки/Плагиат
      S21: Несанкционированные советы
      S22: Незаконная деятельность
      S23: Аморальное/Неэтичное

      Сообщение пользователя: "{{ user_input }}"

      Верни оценку безопасности в JSON формате:
      {"User Safety": "safe" или "unsafe", "Safety Categories": список нарушенных категорий через запятую. Пропусти если все безопасно.}

      Верни только JSON, ничего больше.
    output_parser: json

  - task: self_check_output
    content: |
      Проверь, содержит ли ответ ассистента небезопасный контент согласно нашей политике безопасности.
      Категории небезопасного контента:
      S1: Насилие
      S2: Сексуальный контент
      S3: Планирование преступлений/Признания
      S4: Оружие и незаконное оружие
      S5: Контролируемые/Регулируемые вещества
      S6: Суицид и самоповреждение
      S7: Сексуальный контент (несовершеннолетние)
      S8: Ненависть/Идентификационная ненависть
      S9: PII/Конфиденциальность
      S10: Харассмент
      S11: Угрозы
      S12: Ненормативная лексика
      S13: Требует осторожности
      S14: Другое
      S15: Манипуляции
      S16: Мошенничество/Обман
      S17: Вредоносное ПО
      S18: Высокорисковые государственные решения
      S19: Политика/Дезинформация/Теории заговора
      S20: Авторские права/Товарные знаки/Плагиат
      S21: Несанкционированные советы
      S22: Незаконная деятельность
      S23: Аморальное/Неэтичное

      Сообщение пользователя: "{{ user_input }}"
      Ответ ассистента: "{{ bot_response }}"

      Верни оценку безопасности в JSON формате:
      {"User Safety": "safe" или "unsafe", "Response Safety": "safe" или "unsafe", "Safety Categories": список нарушенных категорий через запятую. Пропусти если все безопасно.}

      Верни только JSON, ничего больше.
    output_parser: json

Writing guardrails/config.yaml


In [22]:
#@title guardrails/rails/simple.co
%%writefile guardrails/rails/simple.co

# Базовые интенты
define user greeting
  "привет"
  "здравствуйте"
  "добрый день"
  "доброе утро"
  "добрый вечер"
  "hello"
  "hi"

define user express goodbye
  "пока"
  "до свидания"
  "всего доброго"
  "goodbye"
  "bye"
  "до встречи"

define user ask about capabilities
  "что ты умеешь"
  "какие у тебя функции"
  "расскажи о себе"

define user ask question
  "кто такой"
  "что такое"
  "как работает"
  "где найти"
  "сколько"
  "когда"
  "почему"

define user any question
  "..."

# Ответы бота
define bot express greeting
  "Здравствуйте! Я ваш ассистент. Помогу найти информацию в базе знаний. Задайте ваш вопрос."

define bot express goodbye
  "До свидания! Обращайтесь если понадобится помощь."

define bot express capabilities
  "Я помогаю находить информацию в корпоративной базе знаний. Задайте вопрос по рабочей теме, и я найду ответ в документах."

define bot refuse unsafe content
  "Извините, я не могу обработать этот запрос по соображениям безопасности."

# Основные флоу
define flow greeting
  user greeting
  bot express greeting

define flow goodbye
  user express goodbye
  bot express goodbye

define flow capabilities
  user ask about capabilities
  bot express capabilities

define flow rag_flow
  user any question
  $answer = execute generate_rag_answer
  bot $answer

define flow unsafe_input
  user ...
  bot refuse unsafe content

Writing guardrails/rails/simple.co


#### Main

In [23]:
# @title Переменные приложения

####################################

# Порты для сервисов
PORT_FLASK = 5000
PHOENIX_PORT = 6006
os.environ['PHOENIX_PORT'] = str(PHOENIX_PORT)

# Используемые модели
COMPLITION_MODEL = dict(
    name= "deepseek/deepseek-v3.2-exp-alt",
    temperature= 0.1,
    context_window = 162000,
    max_tokens = 4096,
)
TOOL_MODEL = dict(
    name= "mistralai/mistral-small-3.2-24b-instruct",
    context_window = 128000,

    temperature= 0.0,
    max_tokens = 30,
)

####################################


complition_llm_instance = VseGPT_LLM(
    api_key= os.environ["VSEGPT_API_KEY"],
    model_name= COMPLITION_MODEL["name"],
    context_window= COMPLITION_MODEL["context_window"],
    temperature= COMPLITION_MODEL["temperature"],
    max_tokens= COMPLITION_MODEL["max_tokens"],
    system_prompt='''Ты - полезный ассистент компании. Твоё имя @Agent, если @Agent - это ты или это информация для тебя, следуй ей. Отвечай по делу на русском языке.'''
)
tool_model_instance = VseGPT_LLM(
    api_key= os.environ["VSEGPT_API_KEY"],
    model_name = TOOL_MODEL["name"],
    context_window=TOOL_MODEL["context_window"],
    temperature=TOOL_MODEL["temperature"],
    max_tokens=TOOL_MODEL["max_tokens"],
)

Settings.llm = complition_llm_instance
Settings.embed_model = HuggingFaceEmbedding(
    # model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    # ИЛИ еще лучше:
    model_name="intfloat/multilingual-e5-large"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [24]:
#@title Маршруты

def init_flask_routes(storageManager):

    def error_response(message):
        """Унифицированная функция для возврата ошибок"""
        return app.response_class(
            response=json.dumps({'status': 'error', 'message': message}, ensure_ascii=False),
            mimetype='application/json; charset=utf-8'
        )

    @app.route('/')
    def index():
        return render_template('index.html', **storageManager.get_chashed_info())

    @app.route('/macroses')
    def to_macroses():
        if not storageManager.is_store_inited:
            return error_response('Хранилище не инициализировано')
        return render_template('macroses.html')

    @app.route('/get_macros', methods=['GET'])
    def get_macros():
        if not storageManager.is_store_inited:
            return error_response('Хранилище не инициализировано')
        macro_db = storageManager.get_macro_db()
        return jsonify(macro_db)

    @app.route('/save_macros', methods=['POST'])
    def save_macros():
        if not storageManager.is_store_inited:
            return error_response('Хранилище не инициализировано')
        macros_db = request.get_json()
        storageManager.set_macro_db(macros_db)
        return jsonify({"status": "success"})

    @app.route('/initialize_storage')
    def index_init():
        storageManager.init_clear_store()
        return redirect(url_for('index'))

    @app.route('/export_storage', methods=['GET'])
    def export_storage():
        if not storageManager.is_store_inited:
            return error_response('Хранилище не инициализировано')

        try:
            zip_buffer = storageManager.export_storage_archive()
            return send_file(
                zip_buffer,
                as_attachment=True,
                download_name='vector_store_backup.zip',
                mimetype='application/zip'
            )
        except Exception as e:
            return error_response(f'Ошибка экспорта: {str(e)}')

    @app.route('/import_storage', methods=['POST'])
    def import_storage():
        try:
            if 'file' not in request.files:
                return error_response('Файл не выбран')

            file = request.files['file']
            if file.filename == '':
                return error_response('Файл не выбран')

            if not file.filename.endswith('.zip'):
                return error_response('Только ZIP архивы поддерживаются')

            success = storageManager.import_storage_archive(file.read())

            if success:
                return jsonify({
                    'status': 'success',
                    'message': 'Хранилище успешно импортировано',
                    'documents_count': len(storageManager.documents),
                    'nodes_count': storageManager.node_count,
                    'macros_count': storageManager.macro_count
                })
            else:
                return error_response('Ошибка импорта хранилища')

        except Exception as e:
            return error_response(f'Ошибка импорта: {str(e)}')

    @app.route('/load_documents')
    def load_docs_page():
        if storageManager.is_store_inited:
            return render_template('fileloader.html')
        return error_response('Хранилище не инициализировано')

    @app.route('/scan', methods=['POST'])
    def scan():
        if not storageManager.is_store_inited:
            return error_response('Хранилище не инициализировано')

        path = request.json.get('path', '')
        if not path or not os.path.exists(path):
            return error_response('Путь не существует или недоступен')

        if not os.path.isdir(path):
            return error_response('Указанный путь не является папкой')

        tree = scan_directory(path, None)
        if not tree:
            return error_response('Не удалось просканировать папку')

        return jsonify({'status': 'success', 'tree': tree})

    @app.route('/check_files_to_index', methods=['POST'])
    def collect_files_fiches():
        if not storageManager.is_store_inited:
            return error_response('Хранилище не инициализировано')

        tree_data = request.json.get('tree')
        selected_files = []
        get_selected_files(tree_data, selected_files)

        if len(selected_files) < 1:
            return error_response('Ни выбран ни 1 файл')

        nodes, fiches = collect_preprocess_files_fiches(selected_files, storageManager)
        stamp = storageManager.save_temp_loader_data({'nodes': nodes})
        fiches, node_texts = prepare_fiches_to_send(fiches, storageManager)
        return jsonify({
            'status': 'success',
            'load-uid': stamp,
            'fiches': fiches,
            "node_texts": node_texts
        })

    @app.route('/send_files_to_index', methods=['POST'])
    def index_files():
        if not storageManager.is_store_inited:
            return error_response('Хранилище не инициализировано')

        load_uid = request.json.get('load-uid')
        fiches = request.json.get('fiches')
        loader_temp_data = storageManager.load_temp_loader_data(load_uid)
        updated_nodes, processed_documents = handle_fiches(loader_temp_data['nodes'], fiches, storageManager)
        storageManager.upload_nodes(updated_nodes, processed_documents)

        return jsonify({
            "status": "success",
            "message": f"Загружено в векторное хранилище: {len(updated_nodes)} нод из {len(processed_documents)} документов",
            "stats": storageManager.get_chashed_info()
        })

    @app.route('/query_vector_store', methods=['POST'])
    def query_vector_store():
        if not storageManager.is_store_inited:
            return error_response('Хранилище не инициализировано')

        try:
            query = request.json.get('query', '').strip()
            if not query:
                return error_response("Пустой запрос")

            # Выполняем запрос к векторному хранилищу
            context = storageManager.retrieve_nodes(query)
            nodes = context['retrieved_nodes']

            print(f'\n {context}')
            print(f'\n {nodes}')

            # Форматируем результаты
            formatted_nodes = []
            for node in nodes:
                topics = []
                topic_level = 1
                while True:
                    topic_level += 1
                    topic = f"topic_level_{topic_level}"
                    if topic not in node.metadata:
                        break
                    else:
                        topics.append("#" * topic_level + node.metadata[topic])

                formatted_nodes.append({
                    "id_": node.node.id_,
                    "text": node.node.text,
                    "file_name": node.node.metadata.get('file_name', 'unknown'),
                    "topics": topics,
                    "similarity": node.score if hasattr(node, 'score') else None
                })

            return jsonify({
                "status": "success",
                "query": query,
                "nodes": formatted_nodes,
                "total_found": len(formatted_nodes)
            })

        except Exception as e:
            error_traceback = traceback.format_exc()
            return error_response(f"Ошибка при выполнении запроса: {str(e)}")

    @app.route('/chat', methods=['POST'])
    def chat_with_llm():
        """Запрос к LLM"""
        try:
            data = request.get_json()
            user_message = data.get('message', '').strip()

            if not user_message:
                return jsonify({'status': 'error', 'message': 'Пустой запрос'})

            # Контекст пользователя (можно расширить)
            user_context = {
                "user_id": "user_123",  # В будущем из сессии/авторизации
                "session_id": "session_456"
            }

            print(f"▶️▶️▶️ LLM response {user_message} ▶️▶️▶️")
            guarded_response = asyncio.run(
                guardrails_manager.process_message(user_message)
            )

            print("📨 Guardrails response:", guarded_response)


            results = {
                'status': 'success',
                'response': guarded_response,
                'original_query': user_message,
                'moderation_applied': True  # Флаг что прошло через Guardrails
            }

            return jsonify(results)

        except Exception as e:
            error_traceback = traceback.format_exc()
            return jsonify({'status': 'error', 'message': f'Ошибка: {str(e)}'})



    @app.route('/flask_health')
    def health_check():
        return {'status': 'ok', 'timestamp': time.time()}

    @app.route('/status')
    def service_status():
        """Статус всех сервисов"""

        # Если мы тут, то флакс точно бежит
        flask_status = "running"

        # Проверяем Phoenix
        phoenix_status = "unknown"
        try:
            response = requests.get(f"http://localhost:{PHOENIX_PORT}", timeout=5)
            phoenix_status = "running" if response.status_code == 200 else "error"
        except requests.exceptions.ConnectionError:
            phoenix_status = "not_running"
        except requests.exceptions.Timeout:
            phoenix_status = "timeout"
        except Exception as e:
            phoenix_status = f"error: {str(e)}"

        result = jsonify({
            "services": {
                "flask_app": {
                    "local": f"http://localhost:{PORT_FLASK}",
                    "public": flask_colab_url,
                    "status": flask_status
                },
                "phoenix_tracing": {
                    "local": f"http://localhost:{PHOENIX_PORT}",
                    "public": phoenix_colab_url,
                    "status": phoenix_status
                }
            },
            "storage": storageManager.get_chashed_info()
        })
        return result

In [25]:
# @title Код функций макросов
%%writefile macro_functions.py

import random
from datetime import date, timedelta

def get_services_availability(macro_handler):
    """
    Это функция-заглушка, использующася как пример.
    Возвращает случайное кол-во (от 4 до 12) несуществующих сервисов-примеров и случайно назначает им сосотяние "Готов" или "Недоступен"
    """
    statuses = ["Готов", "Недоступен"]

    services = []
    for i in range(random.randint(4, 12)):
        service_name = f"Сервис_{i+1}"
        service_status = random.choice(statuses)
        services.append(f"{service_name}: {service_status}")

    result = "\n".join(services)
    return result


def get_non_incident_days(macro_handler):
    """
    Это функция-заглушка, используемая как пример.
    Возвращает случайное количество дней (от 0 до 350) представляющее количество дней без происшествий на производстве
    """
    current_date = date.today()
    formatted_date = f"{current_date.day:02d}.{current_date.month:02d}.{current_date.year}"

    non_incident_days = random.randint(0, 350)

    last_incident_day = current_date - timedelta(days=non_incident_days)
    formatted_last_incident = f"{last_incident_day.day:02d}.{last_incident_day.month:02d}.{last_incident_day.year}"

    return (f"Сегодня {formatted_date}. "
            f"Дней без происшествий {non_incident_days} "
            f"(Последний инцидент {formatted_last_incident})")


Writing macro_functions.py


## Запуск приложения

In [ ]:
storageManager = StorageManager()

guardrails_manager = GuardrailsManager(
    storageManager,
    tool_model_instance,
    config_path = '/content/guardrails'
)

app = Flask(__name__)
globals()['flask_app'] = app
init_flask_routes(storageManager)

def start_phoenix():
    """Запуск Phoenix для трассировки LLM"""
    try:
        print("🚀 Запускаем Phoenix...")
        phoenix_session = px.launch_app()
        print("✅ Phoenix запущен успешно")
        return phoenix_session
    except Exception as e:
        print(f"❌ Ошибка запуска Phoenix: {e}")
        return None


if __name__ == '__main__':
    # Найти процесс Flask и завершить его
    os.system(f"kill $(lsof -t -i:{PORT_FLASK})")

    flask_colab_url = globals().get("flask_colab_url", False)
    if not flask_colab_url:
        print("Пробрасываем Flask...")
        flask_colab_url_start = "google.colab.kernel.proxyPort(" + str(PORT_FLASK) + ")"
        flask_colab_url = eval_js(flask_colab_url_start)
        globals()['flask_colab_url'] = flask_colab_url
        print(f" -> {flask_colab_url}")

    app.config["BASE_URL"] = flask_colab_url

    phoenix_colab_url = globals().get('phoenix_colab_url', False)
    if not phoenix_colab_url:
        # Запускаем Phoenix в фоне
        phoenix_thread = threading.Thread(target=start_phoenix)
        phoenix_thread.daemon = True
        phoenix_thread.start()

        # Настраиваем трассировку для LlamaIndex
        try:
            print("Настраиваем трассировку для LlamaIndex...")
            endpoint = f"http://127.0.0.1:{PHOENIX_PORT}/v1/traces"
            tracer_provider = TracerProvider()
            tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
            LlamaIndexInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)
            print("✅ Трассировка LlamaIndex настроена")
        except Exception as e:
            print(f"⚠️  Трассировка LlamaIndex не настроена: {e}")

        # Даем время для запуска Phoenix
        time.sleep(5)

        print("Пробрасываем Phoenix...")
        phoenix_colab_url_start = "google.colab.kernel.proxyPort(" +  str(PHOENIX_PORT) + ")"
        phoenix_colab_url = eval_js(phoenix_colab_url_start)
        globals()['phoenix_colab_url'] = phoenix_colab_url
        print(f" -> {phoenix_colab_url}")

    print("\n" + "="*60)
    print("🌐 ДОСТУП К FLASK:", flask_colab_url)
    print("🔍 ДОСТУП К PHOENIX TRACING:", phoenix_colab_url)
    print("="*60 + "\n")

    print("Запуск Flask приложения...")
    app.run(port=PORT_FLASK, use_reloader=False, debug=True)

In [ ]:
# Ресет ссылок (если proxyPort изменился)
del globals()['phoenix_colab_url']
del globals()['flask_colab_url']